In [1]:
library(Seurat)
library(SeuratDisk)
library(caret)

Attaching SeuratObject

Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Loading required package: ggplot2

Loading required package: lattice



In [2]:
# library(SeuratDisk)
# Convert ( "../scClass_data/pbmcmm_pp_subsample.h5ad" , dest  =  "h5seurat" , overwrite  =  FALSE )
# Convert ("../scClass_data/pbmc68k_v2.h5ad", dest="h5seurat", overwrite=TRUE) #if the file exist, then overwrite

In [ ]:
ref_file <- "../scClass_data/pbmcmm_pp_subsample.h5seurat"
test.file <- "../scClass_data/cordblood_pp.h5seurat"

In [3]:
ref <- LoadH5Seurat(ref_file)

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [4]:
ref[["random"]] <- sample(0:1, 16778, replace=TRUE)
ref.list <- SplitObject(ref, split.by = "random")
ref.list

$`1`
An object of class Seurat 
20729 features across 8397 samples within 1 assay 
Active assay: RNA (20729 features, 0 variable features)

$`0`
An object of class Seurat 
20729 features across 8381 samples within 1 assay 
Active assay: RNA (20729 features, 0 variable features)


In [5]:
for (i in 1:length(ref.list)) {
    ref.list[[i]] <- NormalizeData(ref.list[[i]], verbose = FALSE)
    ref.list[[i]] <- FindVariableFeatures(ref.list[[i]], selection.method = "vst", nfeatures = 2000,verbose = FALSE)
}

In [6]:
ref.anchors <- FindIntegrationAnchors(object.list = ref.list, dims = 1:30)

Computing 2000 integration features

Scaling features for provided objects

Finding all pairwise anchors

Running CCA

Merging objects

Finding neighborhoods

Finding anchors

	Found 20453 anchors

Filtering anchors

	Retained 15448 anchors



In [7]:
ref.integrated <- IntegrateData(anchorset = ref.anchors, dims = 1:30)

Merging dataset 2 into 1

Extracting anchors for merged samples

Finding integration vectors

Finding integration vector weights

Integrating data



In [8]:
ref.integrated <- ScaleData(ref.integrated, verbose = FALSE)

In [9]:
ref.integrated <- RunPCA(ref.integrated, npcs = 30, verbose = FALSE)

In [10]:
ref.integrated <- RunUMAP(ref.integrated, reduction = "pca", dims = 1:30, verbose = FALSE) # ,return.model = TRUE

Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”


In [11]:
# saveRDS(ref.integrated, file = "../scClass_data/ref_integrated.rds")

## Cell type classification

In [12]:
test.dataset <- LoadH5Seurat(test.file)

Validating h5Seurat file

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



In [14]:
ref.anchors <- FindTransferAnchors(reference = ref.integrated, query =test.dataset,dims = 1:30, reference.reduction = "pca")

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 2836 anchors

Filtering anchors

	Retained 243 anchors



In [37]:
ref.integrated$transfer_cell_type <- as.character(ref.integrated$transfer_id) # celltype = transfer_id (int to char)
predictions <- TransferData(anchorset = ref.anchors, refdata = ref.integrated$transfer_cell_type,dims = 1:30)
test.dataset <- AddMetaData(test.dataset, metadata = predictions)

Finding integration vectors

Finding integration vector weights

Predicting cell labels



## Make Confusion  Matrix

In [61]:
label <- c('T-helper cell','cytotoxic T cell','memory B cell','naive B cell',
           'plasma cell','natural killer cell','erythrocyte',
           'megakaryocyte','monocyte','dendritic cell','bone marrow hematopoietic cell')

In [80]:
y.pred <- factor(label[as.integer(test.dataset$predicted.id)+1],levels=label)
y.true <- factor(test.dataset$transfer_cell_type,levels=label)

In [90]:
mat.table <- confusionMatrix(y.true,y.pred)
mat.df <- as.data.frame.matrix(mat.table$table) 
colnames(mat.df) = 0:10
mat.df

0  1     2 3    4  5   6 7 8    9   10  
T-helper cell                  46 41992 0    0 22  15 0 0   66  17   21
cytotoxic T cell                4 22154 0    0  1   3 0 0    0   0    0
memory B cell                   0     0 0    0  0   0 0 0    0   0    0
naive B cell                    1  6078 0 3559  1   0 0 0    3 443    0
plasma cell                     0     0 0    0  0   0 0 0    0   0    0
natural killer cell             0  8191 0    0  1 191 0 0    2   0    0
erythrocyte                     0     0 0    0  0   0 0 0    0   0    0
megakaryocyte                   0     0 0    0  0   0 0 0    0   0    0
monocyte                        0     6 0    1  2   0 0 0 2170 288    0
dendritic cell                  0     1 0    0  0   0 0 0    0  98    0
bone marrow hematopoietic cell  1  1391 0   11 27   6 0 0  799 124 4053

In [91]:
mat.df.percent <- round(mat.df/(rowSums(mat.df)+0.001)*100, digits=1)
mat.df.percent

0   1     2 3    4   5   6 7 8    9    10  
T-helper cell                  0.1  99.6 0  0.0 0.1 0.0 0 0  0.2  0.0  0.0
cytotoxic T cell               0.0 100.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
memory B cell                  0.0   0.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
naive B cell                   0.0  60.3 0 35.3 0.0 0.0 0 0  0.0  4.4  0.0
plasma cell                    0.0   0.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
natural killer cell            0.0  97.7 0  0.0 0.0 2.3 0 0  0.0  0.0  0.0
erythrocyte                    0.0   0.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
megakaryocyte                  0.0   0.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
monocyte                       0.0   0.2 0  0.0 0.1 0.0 0 0 88.0 11.7  0.0
dendritic cell                 0.0   1.0 0  0.0 0.0 0.0 0 0  0.0 99.0  0.0
bone marrow hematopoietic cell 0.0  21.7 0  0.2 0.4 0.1 0 0 12.5  1.9 63.2

In [92]:
y.pred

0   1     2 3    4   5   6 7 8    9    10  
T-helper cell                  0.1  99.6 0  0.0 0.1 0.0 0 0  0.2  0.0  0.0
cytotoxic T cell               0.0 100.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
memory B cell                  0.0   0.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
naive B cell                   0.0  60.3 0 35.3 0.0 0.0 0 0  0.0  4.4  0.0
plasma cell                    0.0   0.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
natural killer cell            0.0  97.7 0  0.0 0.0 2.3 0 0  0.0  0.0  0.0
erythrocyte                    0.0   0.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
megakaryocyte                  0.0   0.0 0  0.0 0.0 0.0 0 0  0.0  0.0  0.0
monocyte                       0.0   0.2 0  0.0 0.1 0.0 0 0 88.0 11.7  0.0
dendritic cell                 0.0   1.0 0  0.0 0.0 0.0 0 0  0.0 99.0  0.0
bone marrow hematopoietic cell 0.0  21.7 0  0.2 0.4 0.1 0 0 12.5  1.9 63.2

In [100]:
print(paste('acc:',sum(y.pred==y.true)/length(y.pred)*100,'%'))

[1] "acc: 35.157807580429 %"
